In [5]:
from srim import TRIM, SR, Ion, Layer, Target
from srim.output import Results
import os
import numpy as np

output_directory = './Results/10KeV_data'
curve_fit_dir = 'curve_fit'
results = Results(output_directory)

In [6]:
results.phonons

### Save to numpy arrays

In [7]:
# Plot 1: Damage Energy vs. Depth
phon = results.phonons
dx_damage = np.mean(np.diff(phon.depth))
energy_damage = (phon.ions + phon.recoils) * dx_damage

damage_energy_x = phon.depth
damage_energy_y = energy_damage / phon.num_ions
np.save(os.path.join(output_directory, curve_fit_dir, 'damage_energy_x.npy'), damage_energy_x)
np.save(os.path.join(output_directory, curve_fit_dir,'damage_energy_y.npy'), damage_energy_y)

# Plot 2: Ionization Energy vs. Depth
ioniz = results.ioniz
dx_ionization = np.mean(np.diff(ioniz.depth))

ionization_x = ioniz.depth
ionization_y_ions = ioniz.ions
ionization_y_recoils = ioniz.recoils

np.save(os.path.join(output_directory, curve_fit_dir,'ionization_x.npy'), ionization_x)
np.save(os.path.join(output_directory, curve_fit_dir,'ionization_y_ions.npy'), ionization_y_ions)
np.save(os.path.join(output_directory, curve_fit_dir,'ionization_y_recoils.npy'), ionization_y_recoils)

# Plot 3: Vacancies vs. Depth
vac = results.vacancy
vacancy_depth = vac.knock_ons + np.sum(vac.vacancies, axis=1)

vacancies_x = vac.depth
vacancies_y = vacancy_depth

np.save(os.path.join(output_directory, curve_fit_dir,'vacancies_x.npy'), vacancies_x)
np.save(os.path.join(output_directory, curve_fit_dir,'vacancies_y.npy'), vacancies_y)

### Curve Fit

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define the fitting function
def fitting_function(x, A, B, C):
    return A * (x ** B) * np.exp(-C * x)

# Load saved data
damage_energy_x = np.load(os.path.join(output_directory, curve_fit_dir,'damage_energy_x.npy'))
damage_energy_y = np.load(os.path.join(output_directory, curve_fit_dir,'damage_energy_y.npy'))

ionization_x = np.load(os.path.join(output_directory, curve_fit_dir,'ionization_x.npy'))
ionization_y_ions = np.load(os.path.join(output_directory, curve_fit_dir,'ionization_y_ions.npy'))
ionization_y_recoils = np.load(os.path.join(output_directory, curve_fit_dir,'ionization_y_recoils.npy'))

vacancies_x = np.load(os.path.join(output_directory, curve_fit_dir,'vacancies_x.npy'))
vacancies_y = np.load(os.path.join(output_directory, curve_fit_dir,'vacancies_y.npy'))

# Fit and plot function
def fit_and_plot(x, y, title, filename):
    # Initial guess for the fitting parameters
    initial_guess = [1, 1, 0.01]
    
    # Remove zero or negative values to avoid issues with the power function
    mask = (x > 0) & (y > 0)
    x = x[mask]
    y = y[mask]

    # Perform curve fitting
    params, _ = curve_fit(fitting_function, x, y, p0=initial_guess, maxfev=10000)
    
    # Generate fitted curve
    fitted_y = fitting_function(x, *params)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, 'b-', label='Original Data')
    plt.plot(x, fitted_y, 'r--', label=f'Fitted: A={params[0]:.2e}, B={params[1]:.2f}, C={params[2]:.2e}')
    plt.xlabel('Depth [Å]')
    plt.ylabel('Value')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_directory, curve_fit_dir, f'{filename}.png'))
    plt.close()
    
    # Save the fitted parameters
    # np.save(f'{filename}_fit_params.npy', params)
    
    return params

# Apply the function to each dataset
damage_params = fit_and_plot(damage_energy_x, damage_energy_y, 'Damage Energy Fit', 'damage_energy_fit')
ionization_ions_params = fit_and_plot(ionization_x, ionization_y_ions, 'Ionization Energy from Ions Fit', 'ionization_ions_fit')
ionization_recoils_params = fit_and_plot(ionization_x, ionization_y_recoils, 'Ionization Energy from Recoils Fit', 'ionization_recoils_fit')
vacancies_params = fit_and_plot(vacancies_x, vacancies_y, 'Vacancies Fit', 'vacancies_fit')

# Display the fitted parameters
print("Fitted Parameters:")
print(f"Damage Energy: A={damage_params[0]:.2e}, B={damage_params[1]:.2f}, C={damage_params[2]:.2e}")
print(f"Ionization (Ions): A={ionization_ions_params[0]:.2e}, B={ionization_ions_params[1]:.2f}, C={ionization_ions_params[2]:.2e}")
print(f"Ionization (Recoils): A={ionization_recoils_params[0]:.2e}, B={ionization_recoils_params[1]:.2f}, C={ionization_recoils_params[2]:.2e}")
print(f"Vacancies: A={vacancies_params[0]:.2e}, B={vacancies_params[1]:.2f}, C={vacancies_params[2]:.2e}")


Fitted Parameters:
Damage Energy: A=2.18e-04, B=0.96, C=6.72e-03
Ionization (Ions): A=2.10e+00, B=0.66, C=6.61e-03
Ionization (Recoils): A=1.16e-02, B=0.91, C=7.14e-03
Vacancies: A=8.69e-04, B=0.92, C=7.29e-03
